# Read data

In [ ]:
import pandas as pd
!./copy-from-pi.sh >> /dev/null
transfer_df = pd.io.parsers.read_csv('transfer-results.csv', parse_dates=["Timestamp"], index_col="Timestamp")
lan_df = pd.io.parsers.read_csv('eth0-results.csv', parse_dates=["Timestamp"], index_col="Timestamp")
print("Number of records - transfer: {} lan: {}".format(len(transfer_df.index), len(lan_df.index)))
print("Last two hours worth of transfer records...")
transfer_df[-12:] # show the last two hours of transfer records

# Last 24 hours

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn
import datetime
import numpy as np

rollingPeriod = 6

now = datetime.datetime.now()
dayago = now - datetime.timedelta(days=1)
merged = pd.merge(transfer_df[dayago : now], lan_df[dayago : now], left_index=True, right_index=True, how="outer")
merged["NormPacketCount"] = merged["PacketCount"] / merged["Duration"]
merged["Download"] = merged["Download"] / 1000
merged["Upload"] = merged["Upload"] / 1000
merged = merged.drop(["Server ID", "Sponsor", "Server Name", "Distance", "Share", "IP Address", "PacketCount", \
                      "Duration"], axis=1)
merged = merged.resample('5min').mean() # align transfer_df, lan_df timestamps; shuffle data back to round time intervals

merged["Upload"] = merged["Upload"].rolling(rollingPeriod).mean()
merged["Download"] = merged["Download"].rolling(rollingPeriod).mean()
merged["Ping"] = merged["Ping"].rolling(rollingPeriod).mean()
merged["NormPacketCount"] = (merged["NormPacketCount"] * 100).rolling(rollingPeriod).mean()

fig = plt.figure(figsize=(9.5, 9))
ax1 = fig.add_subplot(3, 1, 1)
ax2 = fig.add_subplot(3, 1, 2)
ax3 = fig.add_subplot(3, 1, 3)
ax1.plot(merged["Upload"], color='g', linestyle='solid', label="Up (kbps)")
ax1.plot(merged["Download"],color='r', linestyle='solid', label="Down (kbps)")
ax1.set_title('Raw upload and download performance')
ax1.legend(loc='best')
ax2.plot(merged["Ping"], color='b', linestyle='solid', label="Ping (ms)")
ax2.plot(merged["NormPacketCount"], color='k', linestyle='solid', label="LAN activity (packet count per 100s)")
ax2.set_title('Local LAN activity and ping to remote host')
ax2.legend(loc='best')
ax3.plot(merged["Upload"] / merged["NormPacketCount"], color='g', linestyle='solid', label="Up")
ax3.plot(merged["Download"] / merged["NormPacketCount"], color='r', linestyle='solid', label="Down")
ax3.set_title('Upload and download, divided by LAN activity (packet count)')
ax3.legend(loc='best')